### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# GhostNet

*Author: Huawei Noah's Ark Lab*

**Efficient networks by generating more features from cheap operations**

<img src="https://pytorch.org/assets/images/ghostnet.png" alt="alt" width="50%"/>

In [1]:
import torch
model = torch.hub.load('huawei-noah/ghostnet', 'ghostnet_1x', pretrained=True)
model.eval()

Downloading: "https://github.com/huawei-noah/ghostnet/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/huawei-noah/ghostnet/raw/master/ghostnet_pytorch/models/state_dict_73.98.pth" to /root/.cache/torch/hub/checkpoints/state_dict_73.98.pth


GhostNet(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): GhostBottleneck(
        (ghost1): GhostModule(
          (primary_conv): Sequential(
            (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (cheap_operation): Sequential(
            (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
            (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
        )
        (ghost2): GhostModule(
          (primary_conv): Sequential(
            (0): Conv2d(16, 8, kernel_size=(1, 1)

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [2]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


tensor([ 1.2017e-01, -1.2157e+00, -8.6316e-01, -4.4031e-01, -1.4798e-01,
        -6.2174e-02,  1.0053e+00,  1.4616e+00,  1.4725e+00, -5.8533e-01,
        -7.5813e-01,  2.0558e-01, -4.3331e-01, -6.5629e-01, -6.7759e-01,
        -2.2912e-01,  6.6753e-01, -3.9376e-02,  9.1357e-01, -5.2970e-02,
        -4.8870e-01,  9.5270e-02,  8.0659e-01, -4.3592e-01, -8.0729e-01,
         2.0196e-01, -4.8348e-01, -1.7261e-01,  4.1925e-02,  1.5424e+00,
        -3.2623e-01,  3.5553e-01,  3.0908e-02,  3.2748e-01, -5.1211e-01,
        -7.7316e-01, -5.4865e-01, -2.5402e-01,  3.6358e-01,  9.5621e-01,
         8.4304e-02,  4.8824e-01, -5.9983e-01, -3.5944e-01,  2.0398e-01,
         4.2388e-01,  1.1903e-02,  1.3735e-01, -5.5075e-01, -6.8120e-01,
         8.5751e-03,  1.3971e-01,  1.0920e+00, -6.9391e-01,  2.5174e-01,
         4.8245e-01, -6.6205e-01, -1.8027e-02, -3.6690e-01,  6.3752e-01,
         1.2044e+00, -3.8001e-01, -4.2095e-01, -2.1851e-02, -5.1857e-01,
        -4.9487e-01, -2.6857e-01, -2.9963e-01,  1.6

### Model Description

The GhostNet architecture is based on an Ghost module structure which generate more features from cheap operations. Based on a set of intrinsic feature maps, a series of cheap operations are applied to generate many ghost feature maps that could fully reveal information underlying intrinsic features. Experiments conducted on benchmarks demonstrate that the superiority of GhostNet in terms of speed and accuracy tradeoff.

The corresponding accuracy on ImageNet dataset with pretrained model is listed below.

| Model structure | FLOPs       | Top-1 acc   | Top-5 acc   |
| --------------- | ----------- | ----------- | ----------- |
|  GhostNet 1.0x  | 142M        | 73.98       | 91.46       |


### References

You can read the full paper at this [link](https://arxiv.org/abs/1911.11907).